# Greenhouse gas concentrations to emissions

This is a bit harder. No, it's a total pain.

I think I've done a reasonable job based on Nick's code.

First we check monthly then we check annual.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from scipy.interpolate import interp1d
import pickle
from climateforcing.utils import mkdir_p

from fair21.constants.gases import molwt, burden_per_emission, lifetime, gas_list
from fair21.defaults.gases import (
    partition_fraction,
    pre_industrial_concentration,
    natural_emissions_adjustment,
    iirf_0,
    iirf_cumulative,
    iirf_temperature,
    iirf_airborne,
    iirf_horizon
)
from fair21.gas_cycle import (
    calculate_g,
    calculate_alpha,
)
from fair21.gas_cycle.inverse import unstep_concentration

In [ ]:
# would be awesome to now convert this to csv
with open("../data/output/rcmip-fair21-ssp245-concentrations.pkl","rb") as filein:
    concentration = pickle.load(filein)

In [ ]:
# interpolate all to a monthly timestep
t_new = np.arange(1750+1/24, 2501, 1/12)
concentration_monthly = {}

for specie in concentration.keys():
    f = interp1d(np.arange(1750.5, 2501), concentration[specie], fill_value="extrapolate")
    concentration_monthly[specie] = f(t_new)
    
f = interp1d(np.arange(1750.5, 2501), concentration['CO2'], fill_value='extrapolate')
concentration_co2_halfyear = f(np.arange(1750.25, 2501, 0.5))

In [ ]:
# grab some emissions
emissions_rcmip = {}
df = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
for gas in gas_list:
    gas_rcmip_name = gas.replace("-", "")
    emissions_rcmip[gas] = df.loc[
        (df['Scenario']=='ssp245') & (df['Variable'].str.endswith("|"+gas_rcmip_name)) & (df['Region']=='World'), '1750':
    ].interpolate(axis=1).values.squeeze()

# CO2 and N2O units need to behave
emissions_rcmip["CO2"] = emissions_rcmip["CO2"] / 1000
emissions_rcmip["N2O"] = emissions_rcmip["N2O"] / 1000

In [ ]:
# grab indicative temperature projections
df = pd.read_csv('../data/rcmip-phase2/rcmip-phase2-fair162-ssp245-mean-temperature.csv')
ssp245_temperature_rfmip = df['temperature'].values

t_new = np.arange(1750+1/24, 2501, 1/12)
f = interp1d(np.arange(1750.5, 2501), ssp245_temperature_rfmip, fill_value="extrapolate")
ssp245_temperature_monthly = f(t_new)

t_new = np.arange(1750.25, 2501, 0.5)
f = interp1d(np.arange(1750.5, 2501), ssp245_temperature_rfmip, fill_value="extrapolate")
ssp245_temperature_halfyear = f(t_new)

In [ ]:
# emissions = {}
# cumulative_emissions = {}

# for specie in concentration.keys():
#     cumulative_emissions[specie] = 0
#     gas_boxes=0
#     emissions[specie] = np.zeros(751*12)
#     airborne_emissions = 0
    
#     if specie in ("CO2", "CH4"):
#         g0, g1 = calculate_g(lifetime[specie], partition_fraction[specie])
#         for i in range(751*12): 
#             alpha_lifetime = calculate_alpha(
#                 cumulative_emissions[specie],
#                 airborne_emissions,
#                 ssp245_temperature_monthly[i],
#                 iirf_0[specie],
#                 iirf_cumulative[specie],
#                 iirf_temperature[specie],
#                 iirf_airborne[specie],
#                 g0,
#                 g1
#             )

#             emissions[specie][i], gas_boxes, airborne_emissions = unstep_concentration(
#                 concentration_monthly[specie][i],
#                 gas_boxes,
#                 airborne_emissions,
#                 burden_per_emission[specie],
#                 lifetime[specie],
#                 alpha_lifetime=alpha_lifetime,
#                 pre_industrial_concentration=pre_industrial_concentration[specie],
#                 partition_fraction=partition_fraction[specie],
#                 timestep=1/12,
#                 natural_emissions_adjustment=natural_emissions_adjustment[specie]
#             )
#             cumulative_emissions[specie] = np.sum(emissions[specie][:i] * 1/12)

#     else:
#         alpha_lifetime=1
#         for i in range(751*12): 
#             emissions[specie][i], gas_boxes, airborne_emissions = unstep_concentration(
#                 concentration_monthly[specie][i],
#                 gas_boxes,
#                 airborne_emissions,
#                 burden_per_emission[specie],
#                 lifetime[specie],
#                 alpha_lifetime=alpha_lifetime,
#                 pre_industrial_concentration=pre_industrial_concentration[specie],
#                 partition_fraction=partition_fraction[specie],
#                 timestep=1/12,
#                 natural_emissions_adjustment=natural_emissions_adjustment[specie]
#             )
#             cumulative_emissions[specie] = np.sum(emissions[specie][:i] * 1/12)
#     cumulative_emissions[specie] = np.cumsum(emissions[specie] * 1/12)

In [ ]:
# fig, ax = pl.subplots(6, 8, figsize=(16,16))
# for igas, gas in enumerate(gas_list):
#     iy = igas % 8
#     ix = igas // 8
#     ax[ix, iy].plot(np.arange(1750.5, 2501), emissions_rcmip[gas], label='RCMIP')
#     ax[ix, iy].plot(np.arange(1750+1/24, 2501, 1/12), emissions[gas], label='Back-calculated')
#     ax[ix, iy].set_title(gas)
# fig.tight_layout()

## Check with annual time stepping

In [ ]:
emissions = {}
cumulative_emissions = {}

for specie in concentration.keys():
    cumulative_emissions[specie] = 0
    gas_boxes=0
    emissions[specie] = np.zeros(751)
    airborne_emissions = 0
    
    if specie in ("CO2", "CH4"):
        g0, g1 = calculate_g(lifetime[specie], partition_fraction[specie])
        for i in range(751): 
            alpha_lifetime = calculate_alpha(
                cumulative_emissions[specie],
                airborne_emissions,
                ssp245_temperature_rfmip[i],
                iirf_0[specie],
                iirf_cumulative[specie],
                iirf_temperature[specie],
                iirf_airborne[specie],
                g0,
                g1
            )

            emissions[specie][i], gas_boxes, airborne_emissions = unstep_concentration(
                concentration[specie][i],
                gas_boxes,
                airborne_emissions,
                burden_per_emission[specie],
                lifetime[specie],
                alpha_lifetime=alpha_lifetime,
                pre_industrial_concentration=pre_industrial_concentration[specie],
                partition_fraction=partition_fraction[specie],
                timestep=1,
                natural_emissions_adjustment=natural_emissions_adjustment[specie]
            )
            cumulative_emissions[specie] = np.sum(emissions[specie][:i] * 1)
            #print(gas_boxes)

    else:
        alpha_lifetime=1
        for i in range(751): 
            emissions[specie][i], gas_boxes, airborne_emissions = unstep_concentration(
                concentration[specie][i],
                gas_boxes,
                airborne_emissions,
                burden_per_emission[specie],
                lifetime[specie],
                alpha_lifetime=alpha_lifetime,
                pre_industrial_concentration=pre_industrial_concentration[specie],
                partition_fraction=partition_fraction[specie],
                timestep=1,
                natural_emissions_adjustment=natural_emissions_adjustment[specie],
            )
            cumulative_emissions[specie] = np.sum(emissions[specie][:i] * 1)
    cumulative_emissions[specie] = np.cumsum(emissions[specie] * 1)

In [ ]:
# turn into arrays for speed
n_gases = len(gas_list)
n_timesteps = len(concentration["CFC-11"])

In [ ]:
%%time

# initialise arrays. Using arrays makes things run quicker
emissions_array = np.ones((n_timesteps, n_gases)) * np.nan
concentration_array = np.ones((n_timesteps, n_gases)) * np.nan
g0 = np.ones((n_gases)) * np.nan
g1 = np.ones((n_gases)) * np.nan
alpha_lifetime = np.ones((n_gases))
airborne_emissions = np.zeros((n_gases))
gas_boxes = np.zeros((n_gases, 4))  # TODO: 4 needs to be unhardcoded
cumulative_emissions_array = np.zeros((n_timesteps, n_gases))
iirf_0_array = np.ones(n_gases) * np.nan
iirf_cumulative_array = np.ones(n_gases) * np.nan
iirf_temperature_array = np.ones(n_gases) * np.nan
iirf_airborne_array = np.ones(n_gases) * np.nan
burden_per_emission_array = np.ones(n_gases) * np.nan
lifetime_array = np.ones((n_gases, 4)) * np.nan
pre_industrial_concentration_array = np.ones(n_gases) * np.nan
partition_fraction_array = np.zeros((n_gases, 4))
natural_emissions_adjustment_array = np.ones(n_gases) * np.nan

for i, gas in enumerate(gas_list):
    concentration_array[:, i] = concentration[gas]
    g0[i], g1[i] = calculate_g(lifetime[gas], partition_fraction=partition_fraction[gas])
    iirf_0_array[i] = iirf_0[gas]
    iirf_cumulative_array[i] = iirf_cumulative[gas]
    iirf_temperature_array[i] = iirf_temperature[gas]
    iirf_airborne_array[i] = iirf_airborne[gas]
    burden_per_emission_array[i] = burden_per_emission[gas]
    lifetime_array[i, :] = lifetime[gas]
    pre_industrial_concentration_array[i] = pre_industrial_concentration[gas]
    partition_fraction_array[i, :] = partition_fraction[gas]
    natural_emissions_adjustment_array[i] = natural_emissions_adjustment[gas]
    
for i in range(751): 
    alpha_lifetime = calculate_alpha(
        cumulative_emissions_array[i, :],
        airborne_emissions,
        ssp245_temperature_rfmip[i],
        iirf_0_array,
        iirf_cumulative_array,
        iirf_temperature_array,
        iirf_airborne_array,
        g0,
        g1,
    )
    alpha_lifetime[np.isnan(alpha_lifetime)]=1  # CF4 seems to have an issue. Should we raise warning?
    emissions_array[i, :], gas_boxes, airborne_emissions = unstep_concentration(
        concentration_array[i, :],
        gas_boxes,
        airborne_emissions,
        burden_per_emission_array,
        lifetime_array,
        alpha_lifetime=alpha_lifetime[:, None],
        pre_industrial_concentration=pre_industrial_concentration_array,
        timestep=1,
        partition_fraction=partition_fraction_array,
        natural_emissions_adjustment=natural_emissions_adjustment_array[:, None],
    )
    cumulative_emissions_array[None, :] = np.sum(emissions_array[:i, :] * 1, axis=0)
cumulative_emissions_array = np.cumsum(emissions_array, axis=0)

In [ ]:
# # put back together
emissions_from_array = {}
for i, gas in enumerate(gas_list):
    emissions_from_array[gas] = emissions_array[:, i]

In [ ]:
fig, ax = pl.subplots(6, 8, figsize=(16,16))
for igas, gas in enumerate(gas_list):
    iy = igas % 8
    ix = igas // 8
    ax[ix, iy].plot(np.arange(1750.5, 2501), emissions_rcmip[gas], label='RCMIP')
    ax[ix, iy].plot(np.arange(1750.5, 2501, 1), emissions[gas], label='Back-calculated')
    ax[ix, iy].plot(np.arange(1750.5, 2501, 1), emissions_from_array[gas], label='Back-calculated array')
    ax[ix, iy].set_title(gas)
fig.tight_layout()